# Test pytables input data generation

In [1]:
%load_ext autoreload
%autoreload 2

import pyximport
import os
import sys
import importlib
import cProfile
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.abspath('../../'))
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug('This will get logged')

import tables as tb
from tables import File, Table
from tables.file import _checkfilters

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

DEBUG:root:This will get logged
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
from converter.input_converter import Converter
from converter.import_Belenguer_format import convert_file
import converter.py_data_write as py_data_write
import converter.shortest_paths as shortest_paths
from converter import load_instance
from converter import Network
from converter import ShortestPath


Error compiling Cython file:
------------------------------------------------------------
...

    d_np = np.ndarray(shape=(nArcs, nArcs), dtype=np.int32)
    p_np = np.ndarray(shape=(nArcs, nArcs), dtype=np.int32)
    arc_cost_np = np.array(arc_cost, dtype=np.int32)

    cdef int d_np_view[nArcs][nArcs]
                             ^
------------------------------------------------------------

/Users/ejwillemse/dev/mcarptif/converter/shortest_paths.pyx:240:30: Not allowed in a constant expression

Error compiling Cython file:
------------------------------------------------------------
...

    d_np = np.ndarray(shape=(nArcs, nArcs), dtype=np.int32)
    p_np = np.ndarray(shape=(nArcs, nArcs), dtype=np.int32)
    arc_cost_np = np.array(arc_cost, dtype=np.int32)

    cdef int d_np_view[nArcs][nArcs]
                      ^
------------------------------------------------------------

/Users/ejwillemse/dev/mcarptif/converter/shortest_paths.pyx:240:23: Not allowed in a constant expressi

ImportError: Building module converter.shortest_paths failed: ["distutils.errors.CompileError: command 'gcc' failed with exit status 1\n"]

In [ ]:
file = 'temp/Lpr_IF-b-01.txt'
file_path = file

In [ ]:
%%time
conv = Converter(file_path, out_to_in=True, overwrite=True)

In [ ]:
%%time
instance_info = conv.extract_inputs()

In [ ]:
success_array = np.array([np.array(x) for x in instance_info['sucArcL']])
instance_df = pd.DataFrame({'arc_u': instance_info['beginL'], 'arc_v': instance_info['endL'], 'arc_cost': instance_info['travelCostL']})
instance_df['arc_index'] = instance_df.index.values
instance_df['arc_inverse_index'] = instance_info['invArcL']
instance_df['arc_inverse_index'] = instance_df['arc_inverse_index'].fillna(-1).astype(int)
instance_df.loc[instance_df['arc_inverse_index'] == -1, 'arc_oneway'] = True
instance_df.loc[instance_df['arc_inverse_index'] != -1, 'arc_oneway'] = False
instance_df['arc_successor_index_list'] = np.nan
instance_df['arc_successor_index_list'] = instance_df['arc_successor_index_list'].astype(object)
instance_df.at[:, 'arc_successor_index_list'] = success_array

In [ ]:
instance_df2 = convert_file(file)

In [ ]:
pd._testing.array_equivalent(instance_df, instance_df2)

In [ ]:
instance_df2

In [ ]:
instance_df3 = instance_df2[['arc_u', 'arc_v', 'arc_cost']]

In [ ]:
network = Network(instance_df3)

In [ ]:
network.update_network_df()

In [ ]:
pd._testing.array_equivalent(network.network_df, instance_df2[['arc_u', 'arc_v', 'arc_cost', 'arc_index', 'arc_successor_index_list']])

Arc u and arc v could be anything, but when using tables, their type has to be set.

In [ ]:
network.create_pytable('temp/Lpr_IF-b-01_fixed_test.h5', overwrite=False)

In [ ]:
with tb.open_file('temp/Lpr_IF-b-01_fixed_test.h5', 'r') as h5file:
    print(h5file)
    arc_info = h5file.root.arc_info
    arc_cost = arc_info.arc_cost.read()

In [ ]:
arc_cost

In [ ]:
sp_calc = ShortestPath()
sp_calc.load_from_pytable('temp/Lpr_IF-b-01_fixed_test.h5')

In [ ]:
sp_calc.arc_u

In [ ]:
sp_calc.calc_shortest_path_internal()

In [ ]:
sp_calc.sp_arc_distances

In [ ]:
sp_calc.sp_arc_distances[10:20, 10:20]

In [ ]:
info = load_instance(file_path = 'temp/Lpr_IF-b-01.txt')

In [ ]:
d_test = np.array(info.d_full)

In [ ]:
instance_info = convert_file(file)
instance_df = instance_info[['arc_u', 'arc_v', 'arc_cost']]
network = Network(instance_df)
network.create_pytable('temp/Lpr_IF-b-01_fixed_test_play.h5', overwrite=True)

with tb.open_file('temp/Lpr_IF-b-01_fixed_test_play.h5', 'a') as h5file:
    filters = tb.Filters(complevel=1)
    sp_info = h5file.create_group(h5file.root, 'shortest_path_info', 'Shortest path info for network')

    l = h5file.root.arc_info.arc_cost.shape[0]

    cost_matrix = h5file.create_carray(sp_info, 
                                       'cost_matrix', 
                                       tb.Int32Col(), 
                                       shape=(l, l),
                                       title='shortest path cost matrix between all arc indices',
                                       filters=filters)
    predecesor_matrix = h5file.create_carray(sp_info, 
                                             'predecesor_matrix', 
                                             tb.Int32Col(), 
                                             shape=(l, l),
                                             title='shortest path predecessor index matrix between all arc indices',
                                             filters=filters)
    
    cost_matrix[:, :] = sp_calc.sp_arc_distances[:, :]
    predecesor_matrix[:, :] = sp_calc.sp_arc_predecessor[:, :]
    print(h5file)
h5file.close()

In [ ]:
with tb.open_file('temp/Lpr_IF-b-01_fixed_test_play.h5', 'r') as h5file:
    filters = tb.Filters(complevel=1)
    sp_info = h5file.root.shortest_path_info

    cost_matrix = sp_info.cost_matrix
    predecesor_matrix = sp_info.predecesor_matrix
    cost_matrix[:10]
    predecesor_matrix[:10]
    print(h5file)

In [ ]:
file = 'temp/Lpr_IF-b-01' 
instance_info = convert_file('{}.txt'.format(file))
instance_df = instance_info[['arc_u', 'arc_v', 'arc_cost']]
network = Network(instance_df)
network.create_pytable('{}_fixed_test_play.h5'.format(file), overwrite=True)
sp_calc = ShortestPath()
sp_calc.load_from_pytable('{}_fixed_test_play.h5'.format(file))
sp_calc.calc_shortest_path_internal()
sp_calc.store_shortest_path_internal()

In [ ]:
file = 'temp/Lpr_IF-b-01' 
instance_info = convert_file('{}.txt'.format(file))
instance_df = instance_info[['arc_u', 'arc_v', 'arc_cost']]
network = Network(instance_df)
network.create_pytable('{}_fixed_test_play.h5'.format(file), overwrite=True)
sp_calc = ShortestPath()
sp_calc.load_from_pytable('{}_fixed_test_play.h5'.format(file))
with tb.open_file('temp/Lpr_IF-b-01_fixed_test_play.h5', 'r') as h5file:
    filters = tb.Filters(complevel=1)
    sp_info = h5file.root.shortest_path_info

    cost_matrix = sp_info.cost_matrix
    predecesor_matrix = sp_info.predecesor_matrix
    cost_matrix[:10]
    predecesor_matrix[:10]
    print(h5file)

In [ ]:
SP_pytable